<a href="https://colab.research.google.com/github/pytholabsbot1/anime_recommender/blob/master/anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle lightfm

!mkdir /root/.kaggle

!ls /root/.kaggle

open('/root/.kaggle/kaggle.json','w').write('{"username":"vaqasam","key":"2ec5d91c66a678bc314920e84c687f3b"}')
!chmod 600 /root/.kaggle/kaggle.json


    100% |████████████████████████████████| 307kB 12.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [2]:
!kaggle datasets download CooperUnion/anime-recommendations-database

 48% 12.0M/25.2M [00:00<00:00, 23.6MB/s]
100% 25.2M/25.2M [00:00<00:00, 51.3MB/s]


In [3]:
!unzip anime-recommendations-database.zip

Archive:  anime-recommendations-database.zip
  inflating: anime.csv               
  inflating: rating.csv              


In [11]:
!wget https://raw.githubusercontent.com/pytholabsbot1/anime_recommender/master/recsys.py
!wget https://raw.githubusercontent.com/pytholabsbot1/anime_recommender/master/generic_sys.py

--2019-04-05 13:23:41--  https://raw.githubusercontent.com/pytholabsbot1/anime_recommender/master/recsys.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8190 (8.0K) [text/plain]
Saving to: ‘recsys.py’

recsys.py           100%[===================>]   8.00K  --.-KB/s    in 0s      

2019-04-05 13:23:41 (84.2 MB/s) - ‘recsys.py’ saved [8190/8190]

--2019-04-05 13:23:43--  https://raw.githubusercontent.com/pytholabsbot1/anime_recommender/master/generic_sys.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10609 (10K) [text/plain]
Saving to

In [0]:
# Importing Libraries and cookbooks
from recsys import * ## recommender system cookbook
from generic_sys import * ## pre-processing code
from IPython.display import HTML ## Setting display options for Ipython Notebook
import pandas as pd,numpy as np

In [17]:
anime = pd.read_csv('anime.csv')
ratings = pd.read_csv('rating.csv').iloc[:100000]

ids = np.unique(ratings.anime_id)

anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [18]:
anime_dict = {}

for i,el in enumerate(anime.anime_id.values):
  if(el in ids):
    #print(el)
    anime_dict[el] = anime.name.values[i]
    #break
    
print(anime_dict)

{32281: 'Kimi no Na wa.', 5114: 'Fullmetal Alchemist: Brotherhood', 28977: 'Gintama°', 9253: 'Steins;Gate', 9969: 'Gintama&#039;', 32935: 'Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou', 11061: 'Hunter x Hunter (2011)', 820: 'Ginga Eiyuu Densetsu', 15335: 'Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare', 15417: 'Gintama&#039;: Enchousen', 4181: 'Clannad: After Story', 918: 'Gintama', 2904: 'Code Geass: Hangyaku no Lelouch R2', 28891: 'Haikyuu!! Second Season', 199: 'Sen to Chihiro no Kamikakushi', 23273: 'Shigatsu wa Kimi no Uso', 24701: 'Mushishi Zoku Shou 2nd Season', 12355: 'Ookami Kodomo no Ame to Yuki', 1575: 'Code Geass: Hangyaku no Lelouch', 263: 'Hajime no Ippo', 44: 'Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen', 1: 'Cowboy Bebop', 30276: 'One Punch Man', 164: 'Mononoke Hime', 7311: 'Suzumiya Haruhi no Shoushitsu', 17074: 'Monogatari Series: Second Season', 21939: 'Mushishi Zoku Shou', 457: 'Mushishi', 2001: 'Tengen Toppa Gurren Lagann', 245: 'Great T

In [19]:
# Creating interaction matrix using rating data
interactions = create_interaction_matrix(df = ratings,
                                         user_col = 'user_id',
                                         item_col = 'anime_id',
                                         rating_col = 'rating')
interactions.shape

(1028, 5154)

In [0]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [0]:
## Creating item-item distance matrix
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,interactions = interactions)
## Checking item embedding distance matrix
#item_item_dist.head()


In [24]:
## Calling 10 recommended items for item id 
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 1535,
                                    item_dict = anime_dict,
                                    n_items = 10)

Item of interest :Death Note
Item similar to the above item:
1- Code Geass: Hangyaku no Lelouch
2- Elfen Lied
3- Shingeki no Kyojin
4- Another
5- Soul Eater
6- Durarara!!
7- Fullmetal Alchemist
8- Code Geass: Hangyaku no Lelouch R2
9- Steins;Gate
10- Deadman Wonderland


In [0]:
rec_list

[501, 5005, 4773, 1157, 9119, 956, 6583, 4375, 636, 6582]

In [0]:
l = anime.name[ratings.anime_id.unique()]

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [23]:
q = input('anime key: ')

for i in anime_dict:
  if(q in anime_dict[i].lower()):
    print(i,anime_dict[i])

anime key: death
1535 Death Note
28223 Death Parade
14353 Death Billiards
2994 Death Note Rewrite
31 Neon Genesis Evangelion: Death &amp; Rebirth
10444 Digimon Xros Wars: Aku no Death General to Nanatsu no Oukoku


In [0]:
import pickle


with open('/content/gdrive/My Drive/item1.pickle', 'wb') as fle:
    pickle.dump(item_item_dist, fle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
import pickle


with open('/content/gdrive/My Drive/dict.pickle', 'wb') as fle:
    pickle.dump(anime_dict, fle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!sudo cp savefile.pickle "/content/gdrive/My Drive/python/save.pickle"

# Downlaod Images 

In [0]:
import requests as req



In [0]:


res = req.get('https://www.bing.com/images/search?q='+anime_name)
img_link = res.text.split('<a class="thumb" target="_blank" href="')[1].split('"')[0]

img_res = req.get(img_link,stream=True)
ext = img_res.headers['Content-Type'].split('/')[-1]

f = open('img.'+ext,'wb')
for chunk in img_res.iter_content(chunk_size=1024):
  if(chunk):
    f.write(chunk)

f.close()

'jpeg'

In [0]:
for id_ in anime_dict:
  
  try:
    res = req.get('https://myanimelist.net/anime/'+str(id_)).text
    base = 'https://cdn.myanimelist.net/images/anime/'
    img_tag = res.split(base)[1].split('"')[0]
    print(img_tag)
    
  except:
    print(id_)
    break

5/87048.jpg
5114


In [0]:
import os

len(os.listdir('anime_images'))

1106

In [0]:
import os,webAssist as wb

for id_ in anime_dict:
  name = anime_dict[id_].replace('/','_').replace(' ','_')
  
  check = [1 for i in os.listdir('anime_images') if(name in i)]
  if(len(check)==0):
    try:
      wb.imageSearch(name).download(num=1,out_dir="anime_images",fname=name)
    except:
      print('error :' +name)
      
  else:
    print('was existing')
    

was existing
was existing
was existing
was existing
https://www.google.co.in/search?q=Gintama&#039;&source=lnms&tbm=isch
was existing
was existing
was existing
was existing
https://www.google.co.in/search?q=Gintama&#039;:_Enchousen&source=lnms&tbm=isch
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was existing
was ex

In [0]:
!pip install google_images_download

    100% |████████████████████████████████| 911kB 19.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/d2/23/84/3cec6d566b88bef64ad727a7e805f6544b8af4a8f121f9691c
Successfully built google-images-download


In [0]:
!zip -r anime_images.zip anime_images

  adding: anime_images/ (stored 0%)
  adding: anime_images/Slayers_Next0.jpeg (deflated 0%)
  adding: anime_images/Kaichou_wa_Maid-sama!0.jpeg (deflated 0%)
  adding: anime_images/Fullmetal_Alchemist:_Brotherhood0.jpeg (deflated 1%)
  adding: anime_images/Tales_of_Vesperia:_The_First_Strike0.jpeg (deflated 0%)
  adding: anime_images/Kochira_Katsushikaku_Kameari_Kouenmae_Hashutsujo_(TV)0.jpeg (deflated 0%)
  adding: anime_images/Shounen_Onmyouji0.jpg (deflated 1%)
  adding: anime_images/Genshiken_OVA0.jpg (deflated 1%)
  adding: anime_images/Dragon_Ball_Kai:_Mirai_ni_Heiwa_wo!_Goku_no_Tamashii_yo_Eien_ni0.jpeg (deflated 0%)
  adding: anime_images/New_Initial_D_Movie:_Legend_1_-_Kakusei0.jpeg (deflated 0%)
  adding: anime_images/Mobile_Suit_Zeta_Gundam0.jpeg (deflated 1%)
  adding: anime_images/Ghost_in_the_Shell:_Arise_-_Border:4_Ghost_Stands_Alone0.jpeg (deflated 0%)
  adding: anime_images/Magic_Kaito0.jpeg (deflated 0%)
  adding: anime_images/Slam_Dunk0.jpeg (deflated 0%)
  adding: an